In [1]:
from scipy.io import loadmat
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import pandas as pd
from pandas.io.parsers import read_csv
import matplotlib as mpl
import matplotlib.pyplot as plt
import math
import scipy . optimize as opt

from sklearn.model_selection import train_test_split

def carga_csv(filename) :
    valores = read_csv(filename, header=None).values
    return valores

datos = carga_csv("Data/wdbc.csv")
X = datos[:, 2:32]
X = X.astype(float)

y = datos[:, 1]

X_train, X_test, y_train, y_test = train_test_split(X, y)


def sigmoid(z):
    return 1/(1+np.exp(-z))

def gradSigmoide(z):
    return np.multiply(sigmoid(z), 1-sigmoid(z))

def hipotesis(comp, x):
    return sigmoid(x.dot(comp))

def pesosAleatorios(L_in, L_out, ini):
    return np.random.rand(L_out, 1+L_in) * 2 * ini - ini

def gradiente(theta1, theta2, X, y, reg):
    delta1 = np.zeros(theta1.shape)
    delta2 = np.zeros(theta2.shape)
    m = len(y)
    
    for i in range(X.shape[0]):
        ones = np.ones(1)
        a1 = np.hstack((ones, X[i]))
        z2 = a1 @ theta1.T
        a2 = np.hstack((ones, sigmoid(z2)))
        z3 = a2 @ theta2.T
        a3 = sigmoid(z3)

        error3 = a3 - y.iloc[i,:][np.newaxis,:]
        z2 = np.hstack((ones, z2))
        error2 = np.multiply(theta2.T @ error3.T, gradSigmoide(z2).T[:,np.newaxis])
        delta1 = delta1 + error2[1:,:] @ a1[np.newaxis,:]
        delta2 = delta2 + error3.T @ a2[np.newaxis,:]
        
    delta1 = delta1 / m
    delta2 = delta2 / m
    
    delta1[:,1:] = delta1[:,1:] + theta1[:,1:] * reg / m
    delta2[:,1:] = delta2[:,1:] + theta2[:,1:] * reg / m
        
    return np.concatenate((np.ravel(delta1), np.ravel(delta2)))

def funcionCoste(theta1, theta2, X, y, landa, num_etiquetas):
    h = propagacion(X, theta1, theta2)
    m = len(y)
    
    matriz1 = np.multiply(y, np.log(h))
    
    matriz2 = np.multiply(1-y, np.log(1-h))
    
    matriz3 = np.sum(matriz1 + matriz2)
    
    primerSumatorio = np.sum(np.sum(np.power(theta1[:,1:],2), axis = 1))
    segundoSumatotrio = np.sum(np.sum(np.power(theta2[:,1:],2), axis = 1))
    regularizacion = (primerSumatorio + segundoSumatotrio) * landa/(2*m)
    
    return np.sum(matriz3 / (-m)) + regularizacion
    
def backprop (params_rn ,num_entradas ,num_ocultas ,num_etiquetas ,X,y ,reg ):
    y_d = pd.get_dummies(y.flatten())   
    
    theta1 = np.reshape(params_rn[:num_ocultas * (num_entradas+1)] ,(num_ocultas,(num_entradas + 1)))
    theta2 = np.reshape(params_rn[num_ocultas * (num_entradas + 1):],( num_etiquetas, (num_ocultas + 1)))
    
    
    Z = funcionCoste(theta1, theta2, X, y_d, reg, 10)   #Calcula coste
    grad = gradiente(theta1, theta2, X, y_d, reg)       #Calcula gradiente
    return Z, grad


def propagacion(X, theta1, theta2):  #FeedForward
    X = np.c_[np.ones((len(X),1)),X]
    capa1 = hipotesis(theta1.T, X)
    capa1 = np.c_[np.ones((len(capa1),1)),capa1]
    salida = hipotesis(theta2.T, capa1)
    return salida

    

#Inicializar parametros de la red neuronal
num_entradas = 30
num_ocultas = 2   #25
num_etiquetas = 2
reg = 1
num_iteraciones = 200

epsilon = 0.12

#Inicializar pesos con el valor de epsilon
theta1 = pesosAleatorios(num_entradas, num_ocultas,epsilon)
theta2 = pesosAleatorios(num_ocultas, num_etiquetas, epsilon)

#Roll de los pesos iniciales
params_rn = np.concatenate((np.ravel(theta1), np.ravel(theta2)))

#Obtener coste y gradiente para los pesos iniciales
cost, grad = backprop (params_rn ,num_entradas ,num_ocultas ,num_etiquetas ,X_train,y_train,reg)

#Minimizar la funcion coste con el gradiente calculado para obtener los pesos optimos
theta_opt = opt.minimize(fun=backprop, x0=params_rn, args=(num_entradas, num_ocultas, num_etiquetas, X_train, y_train, reg), method='TNC', jac=True, options={'maxiter': num_iteraciones})

#Unroll de los pesos optimos
theta1_opt = np.reshape(theta_opt.x[:num_ocultas * (num_entradas+1)] ,(num_ocultas,(num_entradas + 1)))
theta2_opt = np.reshape(theta_opt.x[num_ocultas * (num_entradas + 1):],( num_etiquetas, (num_ocultas + 1)))

#Calcula la salida de la red para los datos X con los pesos optimos 
output = propagacion(X_test,theta1_opt, theta2_opt)

#Hallar la prediccion para cada ejemplo:

predicciones = output.argmax(axis=1)
#predicciones = predicciones

print("El array de predicciones para los casos de prueba es: \n" + str(predicciones))

y_test = np.array(y_test == 'M').astype(int)

#Calcula porcentaje de aciertos
aciertos = np.mean(predicciones == y_test.flatten()) * 100
print("\nEl porcentaje de aciertos de la prediccion para los casos de prueba de la red neuronal de 3 capas tras el entreno de sus pesos para landa = " + str(reg) + " y " + str(num_iteraciones) + " iteraciones es: " + str(aciertos)+" %.")



El array de predicciones para los casos de prueba es: 
[0 1 1 1 0 0 1 1 0 1 0 1 1 0 1 0 0 1 0 0 1 1 0 0 0 1 0 1 1 0 0 0 0 0 0 1 1
 1 1 1 0 0 0 0 1 0 1 1 1 1 1 0 1 1 0 0 0 1 0 0 1 1 0 1 1 0 0 1 1 1 0 1 0 0
 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 0 0 1 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1
 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1]

El porcentaje de aciertos de la prediccion para los casos de prueba de la red neuronal de 3 capas tras el entreno de sus pesos para landa = 1 y 200 iteraciones es: 95.1048951048951 %.
